# TFG: Ramón Guillermo García Manrique (78.700.574-W)

Reconocimiento facial en el ámbito de la inmigración irregular para determinación de la edad

### Sentencias import

In [1]:
import numpy as np
import face_recognition
import cv2
import os, sys
import datetime
#import mysql.connector

### Reconocimiento facial en tiempo real

In [2]:
# Declaramos distintos arrays para almacenar datos
face_locations = [] # Localizaciones de caras en el frame
face_encodings = [] # Valores de las caras detectadas
face_names = [] 
known_face_encodings = [] # Valores de las caras conocidas
known_face_names = [] # Nombres asociados a las caras conocidas


# Para cada imagen almacenada en el directorio "imagenes" se extraen valores de las caras y nombre del archivo asociado
for image in os.listdir("imagenes"):
     face_image = face_recognition.load_image_file(f"imagenes/{image}")
     face_encoding = face_recognition.face_encodings(face_image)[0]
     known_face_encodings.append(face_encoding)
     known_face_names.append(image)

# El código comentado a continuación guarda la información recabada de las imágenes almacenadas en el directorio imagenes
# en una tabla de una base de datos, y posteriormente recupera dicha información para poder realizar las comparaciones
# con respecto a las líneas anteriores, mete un paso intermedio: insertar en BD y consultar/extraer de BD
"""
conexion = mysql.connector.connect(host="localhost", user="root", passwd="", database="caras1")
cursor = conexion.cursor()
insertar = "insert into caras(medidas, nacimiento) values (%s,%s)"

# Para cada imagen almacenada en el directorio "imagenes" se extraen valores de las caras y nombre del archivo asociado
for image in os.listdir("imagenes"):
     face_image = face_recognition.load_image_file(f"imagenes/{image}")
     face_encoding = face_recognition.face_encodings(face_image)[0]
     datos=(face_encoding.tostring(), image)
     cursor.execute(insertar, datos)

consultar = "select medidas, nacimiento from caras"
cursor.execute(consultar)
result_set = cursor.fetchall()
for row in result_set:
     medidas = row[0]
     known_face_encodings.append(np.fromstring(medidas))
     known_face_names.append(row[1])
conexion.commit()
conexion.close()
"""


###########################################################################################################
# Finalizada declaraciones y recopilación de datos de caras conocidas, comienza la captura en tiempo real #
###########################################################################################################

cap = cv2.VideoCapture(1, cv2.CAP_DSHOW) # El valor 1 indica que se selecciona la cárama frontal de dispositivo (trasera 0)
# Bucle infinito para patura de video en directo
while True:
     ret, frame = cap.read()
     if ret == False: break
     frame = cv2.flip(frame, 1) # Variable referida a la ventana principal de ejecución
     # Carga las localizaciones de las distintas caras que detecte en el frame
     face_locations = face_recognition.face_locations(frame)

     # Si hay localizaciones, es que ha detectado alguna cara, por lo que procede a tratar de indentificarla
     if face_locations != []:
          # Para cada localización (cara) detectada
          for face_location in face_locations:
               face_frame_encodings = face_recognition.face_encodings(frame, known_face_locations=[face_location])[0]
               # Compara valores de cara asoiciada a una localización con todos los valores de caras conocidas
               result = face_recognition.compare_faces(known_face_encodings, face_frame_encodings)
               #print("Result:", result)
               for index, r in enumerate(result):
                    # Se detecta cara conocida
                    if r == True:
                         text = known_face_names[index]
                         fecha, extension = os.path.splitext(text)
                         text = fecha
                         fecha = datetime.datetime.strptime(fecha, '%d-%m-%Y').date()
                         hoy = datetime.date.today()
                         diferencia = hoy - fecha
                         anios = diferencia.days // 365
                         # Mayor de edad, recuadro verde
                         if anios >= 18:
                              color = (125, 220, 0)
                         # Menor de edad, recuadro azul
                         else:
                              color = (255,10,10)
                         break
                    # Cara desconocida, recuadro rojo
                    else:
                         text = "Desconocido"
                         color = (50, 50, 255)
               # Se dibuja rectángulo vacío al rededor de la caras, otro relleno para conformar un letrero, y en el
               # espacio del letrero se aporta dato indetificativo
               cv2.rectangle(frame, (face_location[3], face_location[2]), (face_location[1], face_location[2] + 30), color, -1)
               cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), color, 2)
               cv2.putText(frame, text, (face_location[3], face_location[2] + 20), 2, 0.7, (255, 255, 255), 1)
     cv2.imshow("Frame", frame)
     # Si se pulsa la tecla q, se finaliza el bucle infinito
     if cv2.waitKey(1) == ord('q'):
          break
# Se finaliza la captura y se destruye la ventana
cap.release()
cv2.destroyAllWindows()